# Trending Crypto Hashtag in Tweets

In [1]:
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
import sys
from typing import Iterable, Optional
import requests
import matplotlib.pyplot as plt
import seaborn as sns

def get_sql_context_instance(spark_context):
    if ('sqlContextSingletonInstance' not in globals()):
        globals()['sqlContextSingletonInstance'] = SQLContext(spark_context)
    return globals()['sqlContextSingletonInstance']
        
def updateFunc(new_values: Iterable[int], last_sum: Optional[int]) -> int:
        return sum(new_values) + (last_sum or 0)

def aggregate_tags_count(new_values, total_sum):
    return sum(new_values) + (total_sum or 0)
    
def process_rdd(time, rdd):
    print("----------- %s -----------" % str(time))
    try:
        # Get spark sql singleton context from the current context
        sql_context = get_sql_context_instance(rdd.context)
        # convert the RDD to Row RDD
        row_rdd = rdd.map(lambda w: Row(hashtag=w[0], hashtag_count=w[1]))
        # create a DF from the Row RDD
        hashtags_df = sql_context.createDataFrame(row_rdd)
        # Register the dataframe as table
        hashtags_df.registerTempTable("hashtags")
        # get the top 10 hashtags from the table using SQL and print them
        hashtag_counts_df = sql_context.sql("select hashtag, hashtag_count from hashtags order by hashtag_count desc limit 10")
        hashtag_counts_df.show()
        # call this method to prepare top 10 hashtags DF and send them
    except:
        print("Error: %s" % e)    
        
if __name__ == "__main__":
    sc = SparkContext.getOrCreate();
    ssc = StreamingContext(sc, 5)
    ssc.checkpoint("checkpoint_TwitterApp")
    # read data from port 
    lines = ssc.socketTextStream("localhost",10711) 
    hashtags = lines.flatMap(lambda line: line.split(" "))\
                            .filter(lambda w:'#' in w)\
                          .map(lambda word: (word, 1))#.reduceByKey(lambda x, y: x + y).transform(lambda x: x.sortBy(lambda x: -x[1]))
    tags_totals = hashtags.updateStateByKey(aggregate_tags_count)
    # do processing for each RDD generated in each interval
    tags_totals.foreachRDD(process_rdd)
    
    ssc.start()

----------- 2022-04-28 18:25:05 -----------


/usr/local/spark/python/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


----------- 2022-04-28 18:25:10 -----------
----------- 2022-04-28 18:25:15 -----------


/usr/local/spark/python/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


+---------------+-------------+
|        hashtag|hashtag_count|
+---------------+-------------+
|        #Crypto|            9|
|           #NFT|            6|
|#cryptocurrency|            5|
|        #crypto|            5|
|           #ETH|            4|
|           #XRP|            4|
|           #SOL|            3|
|          #DeFi|            3|
|        #STELLA|            2|
|           #FUD|            2|
+---------------+-------------+

----------- 2022-04-28 18:25:20 -----------
+---------------+-------------+
|        hashtag|hashtag_count|
+---------------+-------------+
|        #Crypto|           17|
|        #crypto|           15|
|#cryptocurrency|           14|
|           #ETH|           10|
|           #NFT|            9|
|          #DeFi|            8|
|           #XRP|            6|
|      #altcoins|            6|
|      #Ethereum|            5|
|    #cryptonews|            4|
+---------------+-------------+

----------- 2022-04-28 18:25:25 -----------
+-------------